In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

2025-09-06 22:03:53.772888: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-06 22:03:55.051230: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
2025-09-06 22:03:57.565203: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [6]:
'''Boilerplate code'''
data = pd.read_csv('Churn_Modelling.csv')

# Preprocessing data

# Drop irrelavant features
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

# Label Encoding on the gender Column
gender_le = LabelEncoder()
data['Gender'] = gender_le.fit_transform(data['Gender'])

# One-hot encode the 'Geography' column
geography_ohe = OneHotEncoder(sparse_output=False) # sparse_output=False is easier with pandas
geo_encoded_array = geography_ohe.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded_array, columns=geography_ohe.get_feature_names_out(['Geography']))

# Combine the new columns and drop the original 'Geography' column
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

# Divide the dataset into independent and dependent features

# since we are predicting customer churn, we'll be keeping exited as the dependent feature and everything else as the independent features
X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features [0-1]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the encoders and scaler for later use
with open('gender_le.pkl', 'wb') as file:
    pickle.dump(gender_le, file)

with open('geography_ohe.pkl', 'wb') as file:
    pickle.dump(geography_ohe, file)

# save the scaler object
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [7]:
## Define a function to create the model and try different parameters (KerasClassifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    
    for _ in range(layers -1): 
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [15]:
# Create a keras classifer
model = KerasClassifier(neurons=16, layers=1,build_fn=create_model, epochs=50, batch_size=10, verbose=1)

In [13]:
# Define the Grid Search parameters
param_grid = {
    'neurons': [16, 32, 64, 128], 
    'layers': [1,2],
    'epochs':[50, 100]
}

In [ ]:
# Perform grid search 
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))


/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/dpxk/miniconda3/envs/dlenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Un